<a href="https://colab.research.google.com/github/ai-fast-track/icevision-gradio/blob/master/icevision_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install git+git://github.com/airctic/mantisshrimp.git#egg=mantisshrimp[inference] --upgrade

In [ ]:
pip install gradio

In [3]:
from mantisshrimp.all import *
import PIL, requests
import torch
from torchvision import transforms
import gradio as gr

In [4]:
MASK_PENNFUNDAN_WEIGHTS_URL = "https://mantisshrimp-models.s3.us-east-2.amazonaws.com/pennfundan_maskrcnn_resnet50fpn.zip"

In [5]:
def load_model(class_map=class_map, url=None):
    if url is None:
        # print("Please provide a valid URL")
        return None
    else:
        model = mask_rcnn.model(num_classes=len(class_map))
        state_dict = torch.hub.load_state_dict_from_url(
            url, map_location=torch.device("cpu")
        )
        model.load_state_dict(state_dict)
        return model

In [ ]:
class_map = datasets.pennfundan.class_map()
model = load_model(class_map=class_map, url=MASK_PENNFUNDAN_WEIGHTS_URL)
# print("class_map: ", class_map)

In [36]:
def predict(
    model, image, detection_threshold: float = 0.5, mask_threshold: float = 0.5
):
    # img = np.array(image)
    tfms_ = tfms.A.Adapter([tfms.A.Normalize()])
    # Whenever you have images in memory (numpy arrays) you can use `Dataset.from_images`
    infer_ds = Dataset.from_images([image], tfms_)

    batch, samples = mask_rcnn.build_infer_batch(infer_ds)
    preds = mask_rcnn.predict(
        model=model,
        batch=batch,
        detection_threshold=detection_threshold,
        mask_threshold=mask_threshold,
    )
    return samples[0]["img"], preds[0]

def get_masks(input_image, display_list, detection_threshold, mask_threshold):
    display_label = ("Label" in display_list)
    display_bbox = ("BBox" in display_list)
    display_mask = ("Mask" in display_list)

    if detection_threshold==0: detection_threshold=0.5
    if mask_threshold==0: mask_threshold=0.5
    
    img, pred = predict(model=model, image=input_image, detection_threshold=detection_threshold, mask_threshold=mask_threshold)
    # print(pred)
    img = draw_pred(img=img, pred=pred, class_map=class_map, denormalize_fn=denormalize_imagenet, display_label=display_label, display_bbox=display_bbox, display_mask=display_mask)
    img = PIL.Image.fromarray(img)
    print("Output Image: ", img.size, type(img))
    return img

In [ ]:
outputs = gr.outputs.Image(type="pil")
display_chkbox = gr.inputs.CheckboxGroup(["Label", "BBox", "Mask"], label="Display")
detection_threshold_slider = gr.inputs.Slider(minimum=0, maximum=1, step=0.1, default=0.5, label="Detection Threshold")

mask_threshold_slider = gr.inputs.Slider(minimum=0, maximum=1, step=0.1, default=0.5, label="Mask Threshold")

gr_interface = gr.Interface(fn=get_masks, inputs=["image", display_chkbox, detection_threshold_slider, mask_threshold_slider], outputs=outputs, title='IceVision - Ice Simple')
gr_interface.launch(inline=False, share=True, debug=True)
